In [2]:
# Some items needed for installation and some needed imports
!pip install qiskit ipywidgets # Installing qiskit for Google Colab
!pip install qiskit-optimization # Needed for an optimzer used (see comment for a specific import)
!pip install cplex # Needed for the CPLEX - it can solve specific types of optimization problems that can be represented as linear or quadratic functions

# This code is related to (not the same) the code made in a separate program, which was an internship I was a part of.
# Importing the mandatory, non-qiskit packages

import numpy as np
import math
import sys
# Just to make sure there are no issues with the python version for the usage of sys.
if sys.version_info < (3, 6):
    raise Exception('Please use Python version 3.6 or greater.')

# this is just to make sure there are no issues no matter which platform is being used.
try:
    import cplex
    from cplex.exceptions import CplexError
except:
    print("Warning: Cplex not found.")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.8 MB 22.0 MB/s 
     |████████████████████████████████| 12.8 MB 71.1 MB/s 
     |████████████████████████████████| 240 kB 65.3 MB/s 
     |████████████████████████████████| 55 kB 3.5 MB/s 
     |████████████████████████████████| 106 kB 53.5 MB/s 
     |████████████████████████████████| 50 kB 7.0 MB/s 
     |████████████████████████████████| 37.5 MB 706 kB/s 
     |████████████████████████████████| 929 kB 72.7 MB/s 
     |████████████████████████████████| 49 kB 5.6 MB/s 
     |████████████████████████████████| 4.0 MB 71.3 MB/s 
     |████████████████████████████████| 1.9 MB 62.1 MB/s 
     |████████████████████████████████| 112 kB 73.5 MB/s 
     |████████████████████████████████| 1.6 MB 52.8 MB/s 
  Created wheel for qiskit: filename=qiskit-0.39.4-py3-none-any.whl size=12274 sha256=25bae938ea50058defc2f504eda378b1090927d3b43b47b141104044d33df363
 

In [3]:
# Creating a class for the classical optimizer
class ClassicalOptimizer:
    def __init__(self, instance,n,v):
      self.instance = instance
      self.n = n  # number of nodes
      self.v = v  # number of vehicles

    def compute_allowed_combinations(self):
      f = math.factorial
      return f(self.n) / f(self.v) / f(self.n-self.v)

    """This uses the classical IBM cplex optimizer. 
    This function and the one right below it was directly based on internship-related code."""

    def cplex_solution(self):
      # refactoring
      instance = self.instance
      n = self.n
      v = self.v
      my_obj = list(instance.reshape(1, n**2)[0])+[0. for x in range(0,n-1)]
      my_ub = [1 for x in range(0,n**2+n-1)]
      my_lb = [0 for x in range(0,n**2)] + [0.1 for x in range(0,n-1)]
      my_ctype = "".join(['I' for x in range(0,n**2)]) + "".join(['C' for x in range(0,n-1)])
      my_rhs = 2*([v] + [1 for x in range(0,n-1)]) + [1-0.1 for x in range(0,(n-1)**2-(n-1))] + [0 for x in range(0,n)]
      my_sense = "".join(['E' for x in range(0,2*n)]) + "".join(['L' for x in range(0,(n-1)**2-(n-1))])+"".join(['E' for x in range(0,n)])
      my_prob = cplex.Cplex()
      self.populatebyrow(my_prob,my_obj,my_ub,my_lb,my_ctype,my_sense,my_rhs)
      my_prob.solve()
      x = my_prob.solution.get_values()
      x = np.array(x)
      cost = my_prob.solution.get_objective_value()
      return x,cost


    def populatebyrow(self,prob,my_obj,my_ub,my_lb,my_ctype,my_sense,my_rhs):
      n = self.n
      prob.objective.set_sense(prob.objective.sense.minimize)
      prob.variables.add(obj = my_obj, lb = my_lb, ub = my_ub, types = my_ctype)
      prob.set_log_stream(None)
      prob.set_error_stream(None)
      prob.set_warning_stream(None)
      prob.set_results_stream(None)
      rows = []
      for ii in range(0,n):
        col = [x for x in range(0+n*ii,n+n*ii)]
        coef = [1 for x in range(0,n)]
        rows.append([col, coef])
      for ii in range(0,n):
        col = [x for x in range(0+ii,n**2,n)]
        coef = [1 for x in range(0,n)]
        rows.append([col, coef])

      # Sub-tour elimination constraints below (from internship code)
      for ii in range(0, n):
        for jj in range(0,n):
          if (ii != jj)and(ii*jj>0):
            col = [ii+(jj*n), n**2+ii-1, n**2+jj-1]
            coef = [1, 1, -1]
            rows.append([col, coef])

        for ii in range(0,n):
          col = [(ii)*(n+1)]
          coef = [1]
          rows.append([col, coef])

        prob.linear_constraints.add(lin_expr=rows, senses=my_sense, rhs=my_rhs)